# Notes on Topics 29 - 32

## Topic 29: Decision Trees
### Intro to Decision Trees
- “recursive binary splitting" The process of training a decision tree and predicting the target features of a dataset is as follows:
    1. Present a dataset of training examples containing features/predictors and a target (similar to classifiers we have seen earlier).
    2. Train the tree model by making splits for the target using the values of predictors. Which features to use as predictors gets selected following the idea of feature selection and uses measures like "information gain" and "Gini Index". We shall cover these shortly.
    3. The tree is grown until some stopping criteria is achieved. This could be a set depth of the tree or any other similar measure.
    4. Show a new set of features to the tree, with an unknown class and let the example propagate through a trained tree. The resulting leaf node represents the class prediction for this example datum.
- CART (Classification and Regression Trees) uses the Gini Index as a metric
- ID3 (Iterative Dichotomiser 3) uses the entropy function and information gain as metrics
- $Entropy(p) = -\sum (P_i . log_2(P_i))$
- High entropy means less predictive power
- As input, the function should take in `D` as a class distribution array for target class, and `a` the class distribution of the attribute to be tested, then calculate gain as $gain(D,A) = Entropy(D) - \sum(\frac{|D_i|}{|D|}.Entropy(D_i))$, where $D_{i}$ represents distribution of each class in `a`

### ID3 Classification Trees: Perfect Split with Information Gain Lab

In [4]:
from math import log
def entropy(pi):
    """
    return the Entropy of a probability distribution:
    entropy(p) = - SUM (Pi * log(Pi) )
    pi is a list of how many occurances there are in each class
    """
    total = 0
    for p in pi:
        p = p / sum(pi)
        if p != 0:
            total +=  p * log(p, 2)
        else:
            total += 0
    total *= -1
    return total
def IG(D, a):
    """
    return the information gain:
    gain(D, A) = entropy(D)− SUM( |Di| / |D| * entropy(Di) )
    """
    total = 0
    for Di in a:
        total += abs(sum(Di) / sum(D)) * entropy(Di)
    gain = entropy(D) - total
    return gain

### Building Trees using scikit-learn + Lab

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
# One-hot encode the training data and show the resulting DataFrame with proper column names
ohe = OneHotEncoder()
ohe.fit(X_train)
X_train_ohe = ohe.transform(X_train).toarray()
# Creating this DataFrame is not necessary its only to show the result of the ohe
ohe_df = pd.DataFrame(X_train_ohe, columns=ohe.get_feature_names(X_train.columns))
ohe_df.head()
# Create the classifier, fit it on the training data and make predictions on the test set
clf = DecisionTreeClassifier(criterion='entropy') # or 'gini'
clf.fit(X_train_ohe, y_train)

In [ ]:
fig, axes = plt.subplots(nrows = 1,ncols = 1, figsize = (3,3), dpi=300)
tree.plot_tree(clf,
               feature_names = ohe_df.columns, 
               class_names=np.unique(y).astype('str'),
               filled = True)
plt.show()

In [ ]:
X_test_ohe = ohe.transform(X_test)
y_preds = clf.predict(X_test_ohe)

# Calculate accuracy 
acc = accuracy_score(y_test,y_pred) * 100
print('Accuracy is :{0}'.format(acc))

# Check the AUC for predictions
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print('\nAUC is :{0}'.format(round(roc_auc, 2)))

# Create and print a confusion matrix 
print('\nConfusion Matrix')
print('----------------')
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

# or
# Alternative confusion matrix
from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(classifier, X, y, values_format='.3g')
plt.show()

### Hyperparameter Tuning and Pruning in Decision Trees + Lab
We can prune our trees using:
- Maximum depth: Reduce the depth of the tree to build a generalized tree. Set the depth of the tree to 3, 5, 10 depending after verification on test data
- Minimum samples leaf with split: minimum number of samples required to split an internal node.
- max_depth and min_samples_split are also both related to the computational cost
- Minimum leaf sample size: minimum number of samples that we want a leaf node to contain. When this minimum size is achieved at a nodE. Size in terminal nodes can be fixed to 30, 100, 300 or 5% of total
- Maximum leaf nodes: Reduce the number of leaf nodes
- Maximum features: Maximum number of features to consider when splitting a node
- The main difference between the two is that min_samples_leaf guarantees a minimum number of samples in a leaf, while min_samples_split can create arbitrary small leaves, though min_samples_split is more common in practice

- For instance, if min_samples_split = 5, and there are 7 samples at an internal node, then the split is allowed. But let's say the split results in two leaves, one with 1 sample, and another with 6 samples. If min_samples_leaf = 2, then the split won't be allowed (even if the internal node has 7 samples) because one of the leaves resulted will have less than the minimum number of samples required to be at a leaf node.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion='entropy', max_features=6, max_depth=3,
                            min_samples_split=0.7, min_samples_leaf=0.25, random_state=SEED)
dt.fit(X_train, y_train)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)

### Regression with CART Trees + Lab
- Recursive partitioning, instead of global model.
- Cost Function: $J(D, \theta) = \frac{n_{left}}{n_{total}} MSE_{left} + \frac{n_{right}}{n_{total}} MSE_{right}$
    - $D$: remaining training examples   
    - $n_{total}$ : number of remaining training examples
    - $\theta = (f, t_f)$: feature and feature threshold
    - $n_{left}/n_{right}$: number of samples in the left/right subset
    - $MSE_{left}/MSE_{right}$: MSE of the left/right subset
- $ \hat{y}_m = \frac{1}{n_{m}} \sum_{i \in D_m} y_i $
$ MSE_m = \frac{1}{n_{m}} \sum_{i \in D_m} (y_i - \hat{y}_m)^2 $
    - $D_m$: training examples in node $m$
    - $n_{m}$ : total number of training examples in node $m$
    - $y_i$: target value of $i-$th example
- Without regularization, decision trees are likely to overfit

In [ ]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state=42, max_depth=3)
regressor.fit(X_train, y_train)
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score
y_pred = regressor.predict(X_test)
print('MAE:', mae(y_test, y_pred))
print('MSE:', mse(y_test, y_pred))
print('RMSE:', np.sqrt(mse(y_test, y_pred)))
print('R-sq score:', r2_score(y_test,y_pred))

### Regression Trees and Model Optimization Lab

- Go to lab to see how to tune parameters manually and plot

In [ ]:
# Create scatter plots for each feature vs. target, can add title, lables, and tight_layout
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 5))
for i, col in enumerate(features.columns):
    plt.subplot(len(features.columns)/3, len(features.columns), i+1)
    plt.plot(data[col], target, 'o')
def performance(y_true, y_predict):
    r2 = r2_score(y_true, y_predict)
    rmse = mean_squared_error(y_true, y_predict, squared=False)
    return [r2, rmse]

## Topic 30: Ensemble Methods

- Bagging, short for Bootstrap Aggregation, is a combination of two ideas -- bootstrap resampling and aggregation
- common approach is to treat each classifier in the ensemble's prediction as a "vote" and let our overall prediction be the majority vote
- also common to see ensembles that take the arithmetic mean of all predictions, or compute a weighted average

### Random Forests
- classification and regression, ensemble of decision trees 
- Bagging: sample two-thirds of our training data with replacement
- the algorithm then uses the remaining one-third of data that wasn't sampled to calculate the Out-Of-Bag Error
- 
- Subspace sampling method: randomly select a subset of features (exact number is tunable parameter) to use as predictors for each node when training a decision tree
- Benefits: Strong performance and interpretability 
- Drawbacks: Computational complexity and memory storage

### Tree Ensembles and Random Forests Lab

In [ ]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
tree_clf = DecisionTreeClassifier(criterion='gini', max_depth=5) # Regular Tree
tree_clf.fit(data_train, target_train)
tree_clf.feature_importances_ # Can plot (In lab)
pred = tree_clf.predict(data_test)
print(confusion_matrix(target_test, pred))
print(classification_report(target_test, pred))
print(accuracy_score(target_test, pred) * 100)
bagged_tree =  BaggingClassifier(DecisionTreeClassifier(criterion='gini', max_depth=5), 
                                 n_estimators=20) # BaggingClassifier
bagged_tree.fit(data_train, target_train)
ran_forest = RandomForestClassifier(n_estimators=100, max_depth= 5) # RandomForestClassifier
ran_forest.fit(data_train, target_train)
forest_2 = RandomForestClassifier(n_estimators = 5, max_features= 10, max_depth= 2)
forest_2.fit(data_train, target_train)

### GridSearchCV + Lab
GridSearchCV: combines K-Fold Cross-Validation with a grid search of parameters
- num models = num_cv + product of parameters
- very time consuming and computationally expensive

In [ ]:
clf = DecisionTreeClassifier()
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [1, 2, 5, 10],
    'min_samples_split': [1, 5, 10, 20]
}
gs_tree = GridSearchCV(clf, param_grid, cv=3)
gs_tree.fit(train_data, train_labels)
gs_tree.best_params_


In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier\
dt_clf = DecisionTreeClassifier()
dt_cv_score = cross_val_score(dt_clf, X_train, y_train, cv=3)
mean_dt_cv_score = np.mean(dt_cv_score)
print(f"Mean Cross Validation Score: {mean_dt_cv_score :.2%}")
dt_param_grid = {'criterion': ['gini', 'entropy'], 'max_depth': [None, 2, 3, 4, 5, 6],
                    'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 3, 4, 5, 6]}
dt_grid_search = GridSearchCV(dt_clf, dt_param_grid, cv=3, return_train_score=True)
dt_grid_search.fit(X_train, y_train)
dt_gs_training_score = np.mean(dt_grid_search.cv_results_['mean_train_score'])
dt_gs_testing_score = dt_grid_search.score(X_test, y_test)
print(f"Mean Training Score: {dt_gs_training_score :.2%}")
print(f"Mean Test Score: {dt_gs_testing_score :.2%}")
print("Best Parameter Combination Found During Grid Search:")
dt_grid_search.best_score_
dt_grid_search.best_params_

### Gradient Boosting and Weak Learners + Lab
Boosting works as follows:
1. Train a single weak learner
2. Figure out which examples the weak learner got wrong
3. Build another weak learner that focuses on the areas the first weak learner got wrong
4. Continue this process until a predetermined stopping condition is met, such as until a set number of weak learners have been created, or the model's performance has plateaued

Adaboost:
- Trained on subset of training sample w/ replacement like bagging, except each data point carries a weight. Weight increases when weak longer misclassifies. Each weight acts as a probability that sample will go into bag

Gradient Boosted Trees:
- Starts with a weak learning and then calculates the Residuals for each data point.
- Model combines the Residuals with a differentiable loss function to calculate the overall loss
- Use gradients and the loss as predictors to train the next tree against. 
- Next learner focuses on these harder examples. Loss is reduced because these more commonly misclassified data points are focused on.
- Gamma, learning rate


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
adaboost_clf = AdaBoostClassifier(random_state=42)
gbt_clf = GradientBoostingClassifier(random_state=42)
adaboost_train_preds = adaboost_clf.predict(X_train)
adaboost_test_preds = adaboost_clf.predict(X_test)
gbt_clf_train_preds = gbt_clf.predict(X_train)
gbt_clf_test_preds = gbt_clf.predict(X_test)
def display_acc_and_f1_score(true, preds, model_name):
    acc = accuracy_score(true, preds)
    f1 = f1_score(true, preds)
    print("Model: {}".format(model_name))
    print("Accuracy: {}".format(acc))
    print("F1-Score: {}".format(f1))
adaboost_confusion_matrix = confusion_matrix(y_test, adaboost_test_preds)
gbt_confusion_matrix = confusion_matrix(y_test, gbt_clf_test_preds)
adaboost_classification_report = classification_report(y_test, adaboost_test_preds)
gbt_classification_report = classification_report(y_test, gbt_clf_test_preds)
print('Mean Adaboost Cross-Val Score (k=5):',
      cross_val_score(adaboost_clf, df, target, cv=5).mean())
print('Mean GBT Cross-Val Score (k=5):',
      cross_val_score(gbt_clf, df, target, cv=5).mean())

### XGBoost
- parallelizes the construction of trees across all your computer's CPU cores during the training phase. It also allows for more advanced use cases
- automatically handles missing values
- `conda install py-xgboost`
- [XGBoost](https://xgboost.readthedocs.io/en/latest/)
- [XG Boost Params](https://xgboost.readthedocs.io/en/latest/parameter.html)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
clf = XGBClassifier()
clf.fit(X_train, y_train)
training_preds = clf.predict(X_train)
test_preds = clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
test_accuracy = accuracy_score(y_test, test_preds)
# Tuning
param_grid = {'learning_rate': [0.1, 0.2], 'max_depth': [6],
    'min_child_weight': [1, 2], 'subsample': [0.5, 0.7], 'n_estimators': [100]}
grid_clf = GridSearchCV(clf, param_grid, scoring='accuracy', cv=None, n_jobs=1)
grid_clf.fit(X_train, y_train)
best_parameters = grid_clf.best_params_
print('Grid Search found the following optimal parameters: ')
for param_name in sorted(best_parameters.keys()):
    print('%s: %r' % (param_name, best_parameters[param_name]))
training_preds = grid_clf.predict(X_train)
test_preds = grid_clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
test_accuracy = accuracy_score(y_test, test_preds)

## Topic 31: Support Vector Machines
- Max Margin classifier: aim to maximize the margin
    - We're trying to solve:
        - $ w_T x^{(i)} + b \geq 1$  if $y ^{(i)} = 1$
        - $ w_T x^{(i)} + b \leq -1$  if $y ^{(i)} = -1$
        - the objective function to maximize: $\dfrac{2}{\lVert w \rVert}$, or minimize $\lVert w \rVert$
    - $w_T$ term is the **weight vector**, $b$ term is called the **bias** 
- Soft Margin classifier:
    - $ b + w_Tx^{(i)} \geq 1-\xi^{(i)}$ if $y ^{(i)} = 1$
    - $ b + w_Tx^{(i)} \leq -1+\xi^{(i)}$ if $y ^{(i)} = -1$
    - the objective function to minimize: $\dfrac{1}{2}\lVert w \rVert^2+ C(\sum_i \xi^{(i)})$
    - $\xi^{(i)}$ is a **slack variable**
- Hyperplane defined by weight vector $w_T$ and bias $b$

- standardize the data

- [kernels](https://scikit-learn.org/stable/modules/svm.html#kernel-functions)
- [svm](https://scikit-learn.org/stable/modules/svm.html)
- [svm.SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)
- [svm.LinearSVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html)
- [svm.NuSVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.NuSVC.html)

In [5]:
# from scratch lab shows how to plot boundaries

### Building an SVM using scikit-learn Lab
- See lab to plot

In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')
clf.fit(X_1, y_1)
clf.coef_
clf.support_vectors_

If you dig deeper into scikit-learn, you'll notice that there are several ways to create linear SVMs for classification:

- `SVC(kernel = "linear")` , what you've used above. Documentation can be found [here](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)  
- `svm.LinearSVC()`, which is very similar to the simple SVC method, but:
    - Does not allow for the keyword "kernel", as it is assumed to be linear (more on non-linear kernels later)
    - In the objective function, `LinearSVC` minimizes the squared hinge loss while `SVC` minimizes the regular hinge loss 
    - `LinearSVC` uses the one-vs-all (also known as one-vs-rest) multiclass reduction while `SVC` uses the one-vs-one multiclass reduction (this is important only when having > 2 classes!)
- `svm.NuSVC()`, which is again very similar,
    - Does have a "kernel" argument
    - `SVC` and `NuSVC` are essentially the same thing, except that for `NuSVC`, `C` is reparametrized into `nu`. The advantage of this is that where `C` has no bounds and can be any positive number, `nu` always lies between 0 and 1  
    - One-vs-one multiclass approach 
    
    
So what does one-vs-one mean? What does one-vs-all mean? 

- One-vs-one means that with $n$ classes, $\dfrac{(n)*(n-1)}{2}$ boundaries are constructed! 
- One-vs-all means that when there are $n$ classes, $n$ boundaries are created.

The difference between these three types of classifiers is mostly small but generally visible for datasets with 3+ classes. Have a look at our third example and see how the results differ!

### The Kernel Trick + Lab
- idea behind [kernel](https://scikit-learn.org/stable/modules/svm.html#kernel-functions) methods is to create (nonlinear) combinations of the original features and project them onto a higher-dimensional space
- some kernels have additional parameters that can be specified and knowing how these parameters work is critical to tuning SVMs
- linear: represented by the inner product of the $\langle x, x' \rangle$
- Radial Basis Function (RBF): $\exp{(-\gamma \lVert x - x' \rVert^2)} $
    - The parameter $C$ is common to all SVM kernels. Again, by tuning the $C$ parameter when using kernels, you can provide a trade-off between misclassification of the training set and simplicity of the decision function. A high $C$ will classify as many samples correctly as possible (and might potentially lead to overfitting)
    - gamma, $\gamma$, defines how much influence a single training example has. The larger $\gamma$ is, the closer other examples must be to be affected

- polynomial: $(\gamma \langle x - x' \rangle+r)^d $
    - $d$ can be specified by the parameter degree. The default degree is 3.
    - $r$ can be specified by the parameter coef0. The default is 0.
- sigmoid: $\tanh ( \gamma\langle x - x' \rangle+r) $

- NuSVC: similar to SVC, but adds an additional parameter, $\nu$, which controls the number of support vectors and training errors. $\nu$ jointly creates an upper bound on training errors and a lower bound on support vectors.
    - Just like SVC, NuSVC implements the "one-against-one" approach when there are more than 2 classes. This means that when there are n classes, $\dfrac{n*(n-1)}{2}$ classifiers are created, and each one classifies samples in 2 classes.

- LinearSVC: similar to SVC, but instead of the "one-versus-one" method, a "one-vs-rest" method is used. So in this case, when there are $n$ classes, just $n$ classifiers are created, and each one classifies samples in 2 classes, the one of interest, and all the other classes. This means that SVC generates more classifiers, so in cases with many classes, LinearSVC actually tends to scale better.

- probability=True to get probability score per class
    - makes the calculations longer because cv needs to be done to calculate probabilities

## Topic 32: ML Pipelines
- Pipelines create an efficient workflow to combine data manipulations, preprocessing, and modeling

[Pipelines](https://www.kdnuggets.com/2017/12/managing-machine-learning-workflows-scikit-learn-pipelines-part-1.html)

[Integrating Grid Search](https://www.kdnuggets.com/2018/01/managing-machine-learning-workflows-scikit-learn-pipelines-part-2.html)

### Pipelines in sklearn Lab

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
# defining the sequence of actions to perform
pipe = Pipeline([('mms', MinMaxScaler()),
                 ('tree', DecisionTreeClassifier(random_state=123))])
# fit the model,
pipe.fit(X_train, y_train)
# score the model on test data
pipe.score(X_test, y_test)

# or implement GridSearch
grid = [{'tree__max_depth': [None, 2, 6, 10], 
         'tree__min_samples_split': [5, 10]}]
gridsearch = GridSearchCV(estimator=pipe, 
                          param_grid=grid, 
                          scoring='accuracy', 
                          cv=5)
gridsearch.fit(X_train, y_train)
gridsearch.score(X_test, y_test)

In [ ]:
# Build a pipeline with StandardScaler and KNeighborsClassifier
scaled_pipeline_1 = Pipeline([('ss', StandardScaler()), 
                              ('knn', KNeighborsClassifier())])
scaled_pipeline_1.fit(X_train, y_train)
scaled_pipeline_1.score(X_test, y_test)

scaled_pipeline_2 = Pipeline([('ss', StandardScaler()), 
                              ('RF', RandomForestClassifier(random_state=123))])
grid = [{'RF__max_depth': [4, 5, 6], 
         'RF__min_samples_split': [2, 5, 10], 
         'RF__min_samples_leaf': [1, 3, 5]}]
gridsearch = GridSearchCV(estimator=scaled_pipeline_2, 
                          param_grid=grid, 
                          scoring='accuracy', 
                          cv=5)
gridsearch.fit(X_train, y_train)
gridsearch.score(X_test, y_test)
gridsearch.best_params_


## Extra Notes
[scoring parameters](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)


In [ ]:
plt.figure(figsize=(20, 10))
plt.boxplot([df[col] for col in df.columns])
plt.title("Box plot of all columns in dataset")
plt.xticks(range(len(df.columns.values)), df.columns.values)